In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [118]:
#3a 
#using nrows rows (10000 for final analysis, 1000 while setting up)
nrows = 10000
# df_old = pd.read_csv("train.csv")[:nrows]
df = pd.read_csv("train.csv")[:nrows]



In [119]:
#3c imputation

#It could be useful to have a variable indicating whether the variable X_f had been imputed for a given row,
#depending on the underlying data. For example, it might be the case that missing values for a survey question about income
#are more likely for respondents with lower incomes--the missing values represent some significant bias, not just random loss.
#however, if it is fairly certain that missing values are mostly due to e.g. data-entry errors, or are relatively few,
#then adding an imputation indicator may be either useless or even somewhat harmful, as it could contribute to ovefitting.



from sklearn.preprocessing import Imputer
#impute mean for integer
copyimputed = False
imp_mean = Imputer(missing_values='NaN', strategy='mean', axis=0,copy=copyimputed)
imp_mode = Imputer(missing_values='NaN', strategy='most_frequent', axis=0, copy=copyimputed )

categorical_vars = []
#will use this list to make dummies for all categorical variables

def categorical_to_int(cat_col):
    int_col = []
    catmap = {}
    currind = 0
    for val in cat_col:
        if val not in catmap.keys():
            catmap[val] = currind
            int_col.append(currind)
            currind += 1
        else:
            int_col.append(catmap[val])
    return int_col


for col in df.columns:
    if(df[col].dtype == np.float64 or  df[col].dtype == np.int64):
        df[col] = imp_mean.fit_transform(df[col].reshape(-1,1))    
    else:
        categorical_vars.append(col)
        #need to convert to integer column for Imputer to work
        df[col] = categorical_to_int(df[col])
        df[col] = imp_mode.fit_transform(df[col].reshape(-1,1))  

In [120]:
# some checks / ways to get an understanding of the data
# print(df['Product_Info_4'].dtype == np.float64)
# df['Product_Info_2'][:10]
# print(categorical_vars)

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
5    4.0
6    5.0
7    4.0
8    0.0
9    2.0
Name: Product_Info_2, dtype: float64


Product_Info_2: 0


In [103]:
# from collections import Counter
# dfcol = []
# for col in df.columns[1:]:
#     dfcol = df[col]

#     if(dfcol.dtype == np.int64):
#         print(col)
#         print(Counter(dfcol.tolist()))

['Product_Info_2']


In [105]:
#3d 

#If all dummy variables are used, the input matrix X will not be full-rank, 
# as any dummy variable column can be constructed as a linear combination of the others,
#   the definition of indicator variables leads to the simple element-level formula x^(i)_j = 1 - sum_{l \neq i| (x^(l)_j) 
#   which implies that on a column level, x^(i) = [1,1,1,...,1] - sum_{l \neq i} x^(l)



for catvar in categorical_vars:
    #create dataframe of dummy vars
    df_toadd = pd.get_dummies(df[catvar], prefix = catvar)
    #add to dataframe
    df = df.join(df_toadd)
    #remove source variable
    df = df.ix[:, df.columns != catvar]

print(df.columns)


Index(['Id', 'Product_Info_1', 'Product_Info_3', 'Product_Info_4',
       'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Ins_Age', 'Ht',
       'Wt',
       ...
       'Product_Info_2_B2', 'Product_Info_2_C1', 'Product_Info_2_C2',
       'Product_Info_2_C3', 'Product_Info_2_C4', 'Product_Info_2_D1',
       'Product_Info_2_D2', 'Product_Info_2_D3', 'Product_Info_2_D4',
       'Product_Info_2_E1'],
      dtype='object', length=146)


In [106]:
#3e
from sklearn.linear_model import LogisticRegression
from sklearn.learning_curve import validation_curve
from quadratic_kappa_for_notebook import qwk
from sklearn.metrics import make_scorer

#convert qwk to a scorer
#idea from the student's answer for the piazza question at https://piazza.com/class/imcf964oqbo6qm?cid=74
my_scorer = make_scorer(qwk, greater_is_better=True) 

X = np.asarray(df.ix[:, df.columns != 'Response'])
# ensure no NaNs left
# print(X[:3])
# for col in X:
#     for val in col:
#         if np.isnan(val):
#             print("na in :" + str(col) + ", " + str(val))

y = df['Response'].tolist()

In [107]:
Crange_init = [.1,.3,1.,10.,100.,1000.]
#used with 1000-row dataset
#yielded test scores of [0.9392, 0.9430, 0.9431, 0.9449, 0.9426, 0.9419]
#best value = 10, but 1 and 100 are reasonably similar
#will try to optimize a bit by looking more in this range

Crange = [.1,.3,1.,3.,7.,10.,20.,30.,100.,300.,1000.]
#with 10k-row dataset, best value was 20 with qwk = .9481

reg = LogisticRegression()
train_scores, test_scores = validation_curve(reg, X, y, "C", Crange, scoring = my_scorer, cv =5)

In [108]:
train_scores_avg = []
test_scores_avg = []
for i in range(0,len(train_scores)):
    train_scores_avg.append(np.mean(train_scores[i]))
    test_scores_avg.append(np.mean(test_scores[i]))
print(train_scores_avg)
print(test_scores_avg)


[0.94599755154788467, 0.94830891819096741, 0.94899781670115213, 0.94953913886064834, 0.94871154543722425, 0.94854154608625585, 0.95104928652677612, 0.94934549318112249, 0.94868273583430318, 0.94679285611639796, 0.94916156478866021]
[0.9440142399068705, 0.94654604007059517, 0.94732834721034853, 0.94706143370129747, 0.94644707114959981, 0.94676909848469604, 0.94812324041873774, 0.94717162589745207, 0.9460350619815564, 0.9443838174894521, 0.94619717848141449]


In [109]:
plt.plot(np.log(Crange), train_scores_avg,'ro')
plt.plot(np.log(Crange),test_scores_avg,'go')
plt.xlabel('ln(C)')
plt.ylabel('Quadratic-Weighted Kappa')
plt.show()

In [110]:
#3f
from sklearn.linear_model import ElasticNet
from sklearn.grid_search import GridSearchCV

In [111]:


# l1_ratios = [.0003,.001,.01,.1]
# alphas = [.003,.01,.1,10.]
#best pre .003 was .1,.01, with score .946
#best post was .1,.003, with .948

# l1_ratios = [.1,.3,.5,.9]
# alphas = [.001,.003,.01]
#best was .5,.001, with score .9484

l1_ratios = [.4,.5,.6]
alphas = [.0001,.0003,.001,.003]
#best was .5,.001, with ~the same score

search_grid = {'l1_ratio':l1_ratios,'alpha':alphas}

ent = ElasticNet(max_iter = 2000)
gscv = GridSearchCV(ent,search_grid,cv=5, scoring=my_scorer)


In [112]:
gscv.fit(X,y)
print(gscv.best_estimator_)
print(gscv.best_score_)


ElasticNet(alpha=0.0001, copy_X=True, fit_intercept=True, l1_ratio=0.6,
      max_iter=2000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
0.954058271398


In [113]:
#3g
from sklearn.learning_curve import learning_curve

#logistic regression is solidly superior by our metric; we will use it with the optimal C-value of 20.

regstar = LogisticRegression(C=20.)

ltrain_sizes = np.array([ 0.1, 0.33, 0.55, 0.78, 1. ])
sizesout, train_scores_learn, test_scores_learn = learning_curve(regstar, X, y, cv = 5, scoring = my_scorer, train_sizes = ltrain_sizes)


In [114]:
ltrain_scores_avg = []
ltest_scores_avg = []
for i in range(0,len(train_scores_learn)):
    ltrain_scores_avg.append(np.mean(train_scores_learn[i]))
    ltest_scores_avg.append(np.mean(test_scores_learn[i]))
print(ltrain_scores_avg)
print(ltest_scores_avg)
               
               


[0.97161527167848294, 0.95756455116862027, 0.95341656799073937, 0.95131644914058955, 0.94831242837314134]
[0.91516505551867433, 0.94902848163115439, 0.94973656723672373, 0.94889343772327917, 0.94679872312284663]


In [115]:
plt.plot(ltrain_sizes, ltrain_scores_avg,'ro')
plt.plot(ltrain_sizes,ltest_scores_avg,'go')
plt.xlabel('Fraction of Data Used')
plt.ylabel('Quadratic-Weighted Kappa')
plt.title('Learning Curve for Logistic Regression')
plt.show()

In [ ]:
#The training and test qwk curves become very close when the full 10,000 rows are used. 
#This implies that there is very little error due to variance (dependency of the fitted model on the data set chosen); 
# the error is almost entirely due to bias (inability of the model to capture all the attributes of the system).